# Silver Data Cleaning

**Purpose:**  
Clean raw data from the [Bronze](./1_bronze.ipynb) layer, to make a large data asset that contains all data that is of interest to us. This includes but is not limited to renaming columns, consolidating field types, consolidating field values, handling nulls and bad data, deduplicating data, and more.

**What this notebook does:**  
1. **Renames** columns to a common, lowercase, snake_case style  
2. **Tags** each row with its `source` (provenance)  
3. **Deduplicates** any exact‐duplicate records  
4. **Cleans** any data that requires cleaning.

This data will be used when creating [Gold](./3_gold.ipynb), where tailored data assets will be created to efficiently answer specific questions.


For more on Medallion Architecture, see [Databricks Glossary: Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture) (Databricks, n.d.).

---

### References  
Databricks. (n.d.). *Medallion Architecture*. Retrieved May 10, 2025, from https://www.databricks.com/glossary/medallion-architecture


---

## Table of Contents

1. [Setup](#setup)  
   Install required packages and import libraries.  

2. [Configuration](#configuration)  
   Define all file paths, API parameters, and date-column lists in one place for reproducibility.  

3. [Data Cleaning & Standardization](#data-cleaning--standardization)  
   Rename columns, lowercase them, and drop duplicates so all sources share the same schema.  

4. [Data Merging & Harmonization](#data-merging--harmonization)  
   Stack the three sources into one “bronze” table, tag each row by origin, and enforce dtypes.  

5. [Quick Exploratory Checks](#quick-exploratory-checks)  
   Check missing values, unique counts, distributions, date ranges, and monthly trends.  

-----

## 1. Setup

**Purpose:**  
Ensure the environment has all necessary libraries installed and imported.  
- `%pip install -r ../../requirements.txt` installs dependencies. 

> **Note:** we use a project-wide `requirements.txt` for consistency

In [1]:
%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd

## 2. Configuration

Below, we define our Bronze Data Assets, so that we can work with them. We also define maps that will help us process the data and join different dataframes together.

In [8]:
BRONZE_DIR = "../../data-assets/bronze"
BRONZE_FILE_NAME = "{}_df.parquet"

BRONZE_FILES = ["dallas", "san_jose", "soco"]
BRONZE_FILE_PATHS = {
    file: os.path.join(BRONZE_DIR, BRONZE_FILE_NAME.format(file)) for file in BRONZE_FILES 
}
BRONZE_DFS = {
    file: pd.read_parquet(path) for file, path in BRONZE_FILE_PATHS.items()
}

## 3. Data Cleaning & Standardization

**Purpose:**  
Clean up and harmonize column names across sources:  
- Apply a single `COLUMN_MAP` dict.  
- Lowercase everything for consistency.  
- Drop unintended duplicates.  

This ensures downstream steps can assume a uniform schema.

In [9]:
# ─── Data Cleaning ───

# Here we will define the full column mapping:
COLUMN_MAP = {
    # Animal ID
    "AnimalID":      "animal_id",
    "Animal_Id":     "animal_id",
    "Animal ID":     "animal_id",
    # Animal Type
    "AnimalType":    "animal_type",
    "Animal_Type":   "animal_type",
    "Type":          "animal_type",
    # Breed
    "PrimaryBreed":  "breed",
    "Animal_Breed":  "breed",
    "Breed":         "breed",
    # Color
    "PrimaryColor":  "primary_color",
    "Color":         "primary_color",
    # Age
    "Age":           "age",
    # Date of Birth
    "Date Of Birth":   "date_of_birth",
    # Sex
    "Sex":           "sex",
    # Intake fields
    "IntakeType":        "intake_type",
    "Intake_type":       "intake_type",
    "Intake Type":       "intake_type",
    "IntakeCondition":   "intake_condition",
    "Intake_Condition":  "intake_condition",
    "Intake Condition":  "intake_condition",
    "IntakeReason":      "intake_reason",
    "reason":            "intake_reason",
    "IntakeDate":        "intake_date",
    "Intake_Date":       "intake_date",
    "Intake Date":       "intake_date",
    # Outcome fields
    "OutcomeType":       "outcome_type",
    "outcome_type":      "outcome_type",
    "Outcome Type":      "outcome_type",
    "OutcomeDate":       "outcome_date",
    "Outcome_Date":      "outcome_date",
    "Outcome Date":      "outcome_date",
}

# Function to apply the column mapping 
def standardize_columns(df: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Standardize DataFrame column names.

    Parameters
    ----------
    df : pandas.DataFrame
        The raw DataFrame whose columns need standardization to enable better
        analysis.
    mapping : dict
        A dict where keys are original column names (exact match) and
        values are the desired standardized names (snake_case).

    Returns
    -------
    pandas.DataFrame
        A copy of `df` with:
        1. Columns renamed according to `mapping`.
        2. All column names converted to lowercase.
        3. Any duplicate column names (arising when multiple originals map
           to the same new name) removed—only the first occurrence is kept.

    Notes
    -----
    - Columns not present in `mapping` are left unchanged (apart from lowercasing).
    - Renaming happens before lowercasing, so mapping keys are case-sensitive.
    - Dropping duplicate columns avoids collisions in downstream code.
    """
    # Apply the renaming mapping
    df = df.rename(columns=mapping)
    # Convert all column names to lowercase
    df.columns = df.columns.str.lower()
    # Remove duplicate columns, keeping the first occurrence
    df = df.loc[:, ~df.columns.duplicated()]
    return df

# Apply the column mapping to each DataFrame
CLEAN_DFS = {
    source: standardize_columns(df, COLUMN_MAP)
    for source, df in BRONZE_DFS.items()
}

# Uncomment the following lines to do a quick check
# --------------------------------------------------------------
# print("San Jose columns:", san_jose_clean.columns.tolist())
# print("Dallas   columns:", dallas_clean.columns.tolist())
# print("SoCo     columns:", soco_clean.columns.tolist())
# --------------------------------------------------------------

-----

## 4. Data Merging & Harmonization

**Purpose:**  
Combine the cleaned DataFrames into one Bronze-layer table:  
- Tag each row with its source (`san_jose`, `dallas`, or `soco`).  
- Reindex to a common `FINAL_COLUMNS` list.  
- Drop exact duplicates and enforce correct dtypes.  

Result: a single `bronze_df` ready for analysis or Silver-layer transforms.

In [13]:
# ─── Data Merging ───

# Tag each DataFrame with its source to allow for tracking
for source, df in CLEAN_DFS.items():
    df["source"] = source

# Now lets define the final column order for the Bronze Layer
FINAL_COLUMNS = [
    "animal_id",
    "animal_type",
    "breed",
    "primary_color",
    "age",
    "date_of_birth",
    "sex",
    "intake_type",
    "intake_condition",
    "intake_reason",
    "intake_date",
    "outcome_type",
    "outcome_date",
    "source",
]

# This is where we will concatenate the DataFrames
def merge_bronze(final_cols: list, **dfs: pd.DataFrame) -> pd.DataFrame:
    """
    Stack multiple source DataFrames into a single Bronze DataFrame.

    Parameters
    ----------
    dfs : one or more pandas.DataFrame
        Cleaned DataFrames to merge (must share standardized column names).
    final_cols : list of str
        Desired column order for the merged Bronze layer.

    Returns
    -------
    pandas.DataFrame
        The combined Bronze DataFrame with only `final_cols`, in that order.
    """
    combined = pd.concat(dfs, ignore_index=True, sort=False)
    # Keep only the columns we care about in the specified order
    combined = combined.reindex(columns=final_cols)
    return combined

# Merge the DataFrames and drop duplicates
bronze_df = merge_bronze(final_cols=FINAL_COLUMNS, **CLEAN_DFS)
bronze_df = bronze_df.drop_duplicates()

# Display basic statistics
print("Final Bronze shape:", bronze_df.shape)
print(bronze_df["source"].value_counts())

Final Bronze shape: (105597, 14)
source
dallas      65063
soco        30550
san_jose     9984
Name: count, dtype: int64


-----

## 3. Quick Exploratory Checks

**Purpose:**  
Perform lightweight diagnostics to understand your data:  
- **Missing values:** Where do we need imputation or exclusion?  
- **Unique counts:** Which columns are constant vs. highly cardinal?  
- **Distributions:** How do key fields like `intake_type` or `outcome_type` break down?  
- **Date range:** Are you covering the expected time span?  
- **Trends:** Monthly intake counts to spot gaps or seasonality.  

Use these insights to guide further cleaning or deeper EDA.

In [14]:
# ─── Step 6: Quick Exploratory Checks ───

# Missing values per column
print("Missing values:")
print(bronze_df.isna().sum(), "\n")

# Cardinality (distinct counts)
print("Unique values per column:")
print(bronze_df.nunique(), "\n")

# Outcome‐type distribution
print("Outcome types (%):")
print(bronze_df["outcome_type"]
      .value_counts(normalize=True)
      .mul(100)
      .round(1)
      .astype(str) + "%", "\n")

# Intake‐type distribution
print("Intake types (%):")
print(bronze_df["intake_type"]
      .value_counts(normalize=True)
      .mul(100)
      .round(1)
      .astype(str) + "%", "\n")

# Time span of intake dates
print("Intake date range:",
      bronze_df["intake_date"].min().date(),
      "to",
      bronze_df["intake_date"].max().date(),
      "\n")

# Age summary (if numeric)
if "age" in bronze_df.columns:
    # coerce to numeric (some age values may be strings)
    bronze_df["age_num"] = pd.to_numeric(bronze_df["age"], errors="coerce")
    print("Age summary:")
    print(bronze_df["age_num"].describe(), "\n")

#Monthly counts by source (to check seasonality or data gaps)
bronze_df["year_month"] = bronze_df["intake_date"].dt.to_period("M")
monthly_counts = (
    bronze_df
    .groupby(["year_month", "source"])
    .size()
    .unstack(fill_value=0)
)
print("Monthly intake counts by source:")
print(monthly_counts.tail())

Missing values:
animal_id                0
animal_type              0
breed                   39
primary_color        65063
age                  95613
date_of_birth        82476
sex                  65063
intake_type              0
intake_condition         0
intake_reason       105140
intake_date              0
outcome_type           371
outcome_date          1448
source                   0
dtype: int64 

Unique values per column:
animal_id           85432
animal_type             6
breed                1221
primary_color         394
age                    61
date_of_birth        6572
sex                    10
intake_type            23
intake_condition       26
intake_reason          25
intake_date          3983
outcome_type           29
outcome_date         3597
source                  3
dtype: int64 

Outcome types (%):
outcome_type
ADOPTION             28.6%
TRANSFER             14.8%
FOSTER               11.7%
RETURN TO OWNER       9.6%
EUTHANIZED            8.2%
RETURNED TO OWNER  

-----